# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe9501dfc10>
├── 'a' --> tensor([[-1.0591, -0.4599, -0.0611],
│                   [-0.5749, -1.1006, -0.2191]])
└── 'x' --> <FastTreeValue 0x7fe9501df940>
    └── 'c' --> tensor([[-0.4247,  0.1040, -0.3506,  1.0081],
                        [ 0.2966,  0.3849,  0.3890, -0.4316],
                        [-0.2976, -1.9565,  0.6580,  0.6349]])

In [4]:
t.a

tensor([[-1.0591, -0.4599, -0.0611],
        [-0.5749, -1.1006, -0.2191]])

In [5]:
%timeit t.a

64.3 ns ± 0.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe9501dfc10>
├── 'a' --> tensor([[-1.1375,  0.9809,  0.3893],
│                   [-2.4563, -0.1378, -0.8775]])
└── 'x' --> <FastTreeValue 0x7fe9501df940>
    └── 'c' --> tensor([[-0.4247,  0.1040, -0.3506,  1.0081],
                        [ 0.2966,  0.3849,  0.3890, -0.4316],
                        [-0.2976, -1.9565,  0.6580,  0.6349]])

In [7]:
%timeit t.a = new_value

66.2 ns ± 0.145 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0591, -0.4599, -0.0611],
               [-0.5749, -1.1006, -0.2191]]),
    x: Batch(
           c: tensor([[-0.4247,  0.1040, -0.3506,  1.0081],
                      [ 0.2966,  0.3849,  0.3890, -0.4316],
                      [-0.2976, -1.9565,  0.6580,  0.6349]]),
       ),
)

In [10]:
b.a

tensor([[-1.0591, -0.4599, -0.0611],
        [-0.5749, -1.1006, -0.2191]])

In [11]:
%timeit b.a

61.2 ns ± 0.0281 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1215, -0.0349, -1.7538],
               [-0.7431,  1.9959, -0.2554]]),
    x: Batch(
           c: tensor([[-0.4247,  0.1040, -0.3506,  1.0081],
                      [ 0.2966,  0.3849,  0.3890, -0.4316],
                      [-0.2976, -1.9565,  0.6580,  0.6349]]),
       ),
)

In [13]:
%timeit b.a = new_value

481 ns ± 0.137 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

829 ns ± 0.0892 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 23.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 664 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 317 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe95024bee0>
├── 'a' --> tensor([[[-1.0591, -0.4599, -0.0611],
│                    [-0.5749, -1.1006, -0.2191]],
│           
│                   [[-1.0591, -0.4599, -0.0611],
│                    [-0.5749, -1.1006, -0.2191]],
│           
│                   [[-1.0591, -0.4599, -0.0611],
│                    [-0.5749, -1.1006, -0.2191]],
│           
│                   [[-1.0591, -0.4599, -0.0611],
│                    [-0.5749, -1.1006, -0.2191]],
│           
│                   [[-1.0591, -0.4599, -0.0611],
│                    [-0.5749, -1.1006, -0.2191]],
│           
│                   [[-1.0591, -0.4599, -0.0611],
│                    [-0.5749, -1.1006, -0.2191]],
│           
│                   [[-1.0591, -0.4599, -0.0611],
│                    [-0.5749, -1.1006, -0.2191]],
│           
│                   [[-1.0591, -0.4599, -0.0611],
│                    [-0.5749, -1.1006, -0.2191]]])
└── 'x' --> <FastTreeValue 0x7fe95024bfd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 64.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe89b47aa00>
├── 'a' --> tensor([[-1.0591, -0.4599, -0.0611],
│                   [-0.5749, -1.1006, -0.2191],
│                   [-1.0591, -0.4599, -0.0611],
│                   [-0.5749, -1.1006, -0.2191],
│                   [-1.0591, -0.4599, -0.0611],
│                   [-0.5749, -1.1006, -0.2191],
│                   [-1.0591, -0.4599, -0.0611],
│                   [-0.5749, -1.1006, -0.2191],
│                   [-1.0591, -0.4599, -0.0611],
│                   [-0.5749, -1.1006, -0.2191],
│                   [-1.0591, -0.4599, -0.0611],
│                   [-0.5749, -1.1006, -0.2191],
│                   [-1.0591, -0.4599, -0.0611],
│                   [-0.5749, -1.1006, -0.2191],
│                   [-1.0591, -0.4599, -0.0611],
│                   [-0.5749, -1.1006, -0.2191]])
└── 'x' --> <FastTreeValue 0x7fe955265c10>
    └── 'c' --> tensor([[-0.4247,  0.1040, -0.3506,  1.0081],
                        [ 0.2966,  0.3849,  0.3890, -0.4316],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 42.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.0591, -0.4599, -0.0611],
                [-0.5749, -1.1006, -0.2191]],
       
               [[-1.0591, -0.4599, -0.0611],
                [-0.5749, -1.1006, -0.2191]],
       
               [[-1.0591, -0.4599, -0.0611],
                [-0.5749, -1.1006, -0.2191]],
       
               [[-1.0591, -0.4599, -0.0611],
                [-0.5749, -1.1006, -0.2191]],
       
               [[-1.0591, -0.4599, -0.0611],
                [-0.5749, -1.1006, -0.2191]],
       
               [[-1.0591, -0.4599, -0.0611],
                [-0.5749, -1.1006, -0.2191]],
       
               [[-1.0591, -0.4599, -0.0611],
                [-0.5749, -1.1006, -0.2191]],
       
               [[-1.0591, -0.4599, -0.0611],
                [-0.5749, -1.1006, -0.2191]]]),
    x: Batch(
           c: tensor([[[-0.4247,  0.1040, -0.3506,  1.0081],
                       [ 0.2966,  0.3849,  0.3890, -0.4316],
                       [-0.2976, -1.9565,  0.6580,  0.6349]],
         

In [26]:
%timeit Batch.stack(batches)

79 µs ± 297 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.0591, -0.4599, -0.0611],
               [-0.5749, -1.1006, -0.2191],
               [-1.0591, -0.4599, -0.0611],
               [-0.5749, -1.1006, -0.2191],
               [-1.0591, -0.4599, -0.0611],
               [-0.5749, -1.1006, -0.2191],
               [-1.0591, -0.4599, -0.0611],
               [-0.5749, -1.1006, -0.2191],
               [-1.0591, -0.4599, -0.0611],
               [-0.5749, -1.1006, -0.2191],
               [-1.0591, -0.4599, -0.0611],
               [-0.5749, -1.1006, -0.2191],
               [-1.0591, -0.4599, -0.0611],
               [-0.5749, -1.1006, -0.2191],
               [-1.0591, -0.4599, -0.0611],
               [-0.5749, -1.1006, -0.2191]]),
    x: Batch(
           c: tensor([[-0.4247,  0.1040, -0.3506,  1.0081],
                      [ 0.2966,  0.3849,  0.3890, -0.4316],
                      [-0.2976, -1.9565,  0.6580,  0.6349],
                      [-0.4247,  0.1040, -0.3506,  1.0081],
                      [ 0.2966,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 390 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

316 µs ± 325 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
